# Procesamiento de datos de spotify
En este notebook se realizará la consulta y procesamiento de los datos que serán utilizados para el proyecto

In [1]:
import glob
import pandas as pd
import numpy as np

In [2]:
path = "user_streaming_history"
csv_files = glob.glob(path + "/*.json")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_json(file) for file in csv_files)

# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)

In [3]:
big_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51189 entries, 0 to 51188
Data columns (total 21 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   ts                                 51189 non-null  object
 1   username                           51189 non-null  object
 2   platform                           51189 non-null  object
 3   ms_played                          51189 non-null  int64 
 4   conn_country                       51189 non-null  object
 5   ip_addr_decrypted                  51189 non-null  object
 6   user_agent_decrypted               51189 non-null  object
 7   master_metadata_track_name         51186 non-null  object
 8   master_metadata_album_artist_name  51186 non-null  object
 9   master_metadata_album_album_name   51186 non-null  object
 10  spotify_track_uri                  51186 non-null  object
 11  episode_name                       3 non-null      object
 12  epis

## Transformación y limpieza del dataset original

In [4]:
big_df[["ts"]] = big_df[["ts"]].apply(pd.to_datetime)
big_df["year_played"] = big_df["ts"].dt.year
big_df["month_played"] = big_df["ts"].dt.month
big_df["min"] = big_df["ms_played"]/60000
big_df=big_df.rename(columns={"ts": "timestamp", "conn_country": "country", "master_metadata_track_name": "song_name", "master_metadata_album_artist_name": "artist_name"})
column_filtered_df = big_df[["timestamp","year_played","month_played","min","country", "song_name", "artist_name","spotify_track_uri"]]

In [5]:
row_filtered_df = column_filtered_df.dropna()
row_filtered_df = row_filtered_df.loc[row_filtered_df["year_played"]==2023]
row_filtered_df = row_filtered_df.loc[row_filtered_df["min"]>0]

## Agregación por mes

In [6]:
df_by_month = pd.pivot_table(
   row_filtered_df,
   index=["country","month_played"],
   aggfunc={'min': np.sum, 'country': len}
).rename(columns={'country': 'count'})
df_by_month.sort_values(by=['min', 'count'],ascending=False, inplace=True)
df_by_month.reset_index(inplace=True)

In [7]:
df_by_month.head(12)

,country,month_played,count,min
0,CO,5,6029,5773.560783
1,CO,8,4560,5004.908217
2,CO,1,4480,4742.008300
3,CO,6,3082,4125.589583
4,CO,4,3590,4003.579650
5,CO,7,3446,3871.571550
6,CO,10,3049,3808.322417
7,CO,3,2890,3447.676717
8,CO,9,3148,3401.536200
9,CO,12,2188,3198.087300


## Agregación de registros por canción

In [8]:
df_by_song = pd.pivot_table(
   row_filtered_df,
   index=["country","song_name", "artist_name"],
   aggfunc={'min': np.sum, 'country': len}
).rename(columns={'country': 'count'})
df_by_song.sort_values(by=['min', 'count'],ascending=False, inplace=True)
df_by_song.reset_index(inplace=True)

In [9]:
df_by_song.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11338 entries, 0 to 11337
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      11338 non-null  object 
 1   song_name    11338 non-null  object 
 2   artist_name  11338 non-null  object 
 3   count        11338 non-null  int64  
 4   min          11338 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 443.0+ KB


In [10]:
df_by_song_month = pd.pivot_table(
   row_filtered_df,
   index=["country","song_name", "artist_name", "month_played"],
   aggfunc={'min': np.sum, 'country': len}
).rename(columns={'country': 'count'})
df_by_song_month.sort_values(by=['min', 'count'],ascending=False, inplace=True)
df_by_song_month.reset_index(inplace=True)

In [11]:
df_by_song_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21885 entries, 0 to 21884
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       21885 non-null  object 
 1   song_name     21885 non-null  object 
 2   artist_name   21885 non-null  object 
 3   month_played  21885 non-null  int64  
 4   count         21885 non-null  int64  
 5   min           21885 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.0+ MB


## Agregación de registros por artista

In [12]:
df_by_artist = pd.pivot_table(
   row_filtered_df,
   index=["country", "artist_name"],
   aggfunc={'min': np.sum, 'country': len}
).rename(columns={'country': 'count'})
df_by_artist.sort_values(by=['min', 'count'],ascending=False, inplace=True)
df_by_artist.reset_index(inplace=True)

In [13]:
df_by_artist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4118 entries, 0 to 4117
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   country      4118 non-null   object 
 1   artist_name  4118 non-null   object 
 2   count        4118 non-null   int64  
 3   min          4118 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 128.8+ KB


In [14]:
df_by_artist_month = pd.pivot_table(
   row_filtered_df,
   index=["country", "artist_name","month_played"],
   aggfunc={'min': np.sum, 'country': len}
).rename(columns={'country': 'count'})
df_by_artist_month.sort_values(by=['min', 'count'],ascending=False, inplace=True)
df_by_artist_month.reset_index(inplace=True)

In [15]:
df_by_artist_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10330 entries, 0 to 10329
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       10330 non-null  object 
 1   artist_name   10330 non-null  object 
 2   month_played  10330 non-null  int64  
 3   count         10330 non-null  int64  
 4   min           10330 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 403.6+ KB
